# Projet M2 MIAS : Risque de réidentification (Février 2026)

**Auteur :** [Votre Nom]
**Superviseur :** Pr Emmanuel Chazard

## 1. Objectif du projet
Conformément au sujet, l'objectif est d'imaginer et tester des indicateurs quantitatifs du risque de réidentification sur des bases de données de santé.

Nous analysons les 3 critères du G29 (CNIL) :
1.  **Individualisation** : Peut-on isoler un individu ?
2.  **Inférence** : Peut-on déduire une information sensible ?
3.  **Corrélation** : Peut-on lier des ensembles de données (conservation de la structure) ?

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

# Configuration graphique
plt.style.use('seaborn-v0_8')

## 2. Chargement des Données

Chargement de la base de référence et des fichiers variants (`out_direct` et `out_sample`) depuis le répertoire `projets_donnees`.

In [ ]:
# Chemin vers le répertoire des données
DATA_DIR = "projets_donnees"

def load_data(directory):
    data = {}
    
    # 1. Chargement de la base de référence
    ref_path = os.path.join(directory, "connaissances_externes.txt")
    if os.path.exists(ref_path):
        data['reference'] = pd.read_csv(ref_path, sep=None, engine='python')
        print(f"Base référence chargée : {len(data['reference'])} lignes")
    else:
        raise FileNotFoundError(f"Le fichier {ref_path} est introuvable.")

    # 2. Chargement des fichiers variants
    pattern = os.path.join(directory, "out_*.txt")
    files = glob.glob(pattern)
    
    if not files:
        print("Attention : Aucun fichier 'out_*.txt' trouvé.")

    for filepath in files:
        filename = os.path.basename(filepath)
        name_key = filename.replace('.txt', '') # ex: out_direct_10
        try:
            df = pd.read_csv(filepath, sep=None, engine='python')
            data[name_key] = df
            print(f" -> Chargé : {name_key} ({len(df)} lignes)")
        except Exception as e:
            print(f"Erreur lors du chargement de {filename} : {e}")
            
    return data

# Exécution du chargement
datasets = load_data(DATA_DIR)
df_reference = datasets['reference']

# Aperçu des colonnes disponibles
print("Colonnes disponibles :", df_reference.columns.tolist())

## 3. Définition des Indicateurs (G29) et Méthodologie

Nous définissons ici les trois indicateurs demandés par la CNIL (G29), en explicitant la méthode de calcul utilisée pour chacun.

### 3.1 Risque d'Individualisation (Taux de Réidentification)
**Technique utilisée : Simulation d'une "Attaque Procureur" (Prosecutor Attack)**

Cette technique simule un attaquant qui connaît des informations spécifiques sur une personne cible (les quasi-identifiants : Âge, Sexe, Date, etc.) et cherche à savoir si cette personne est unique dans la base de données.

* **Méthodologie :**
    1.  **Isolation des uniques ($k=1$) :** Dans le fichier *publié*, nous regroupons les lignes par combinaison de quasi-identifiants (QI). Nous ne gardons que les lignes qui sont **uniques** (qui n'apparaissent qu'une seule fois).
    2.  **Appariement (Matching) :** Nous effectuons une jointure entre ces lignes uniques et la *base de référence* sur les mêmes colonnes QI.
    3.  **Vérification (Vérité Terrain) :** Nous comparons l'`id_sejour`. Si l'ID du fichier publié correspond à l'ID du fichier de référence, l'individualisation est réussie.
* **Formule :**
    $$\text{Taux} = \frac{\text{Nombre de correspondances uniques correctes}}{\text{Taille totale de la base publiée}} \times 100$$

### 3.2 Risque d'Inférence (Déduction d'attribut sensible)
**Technique utilisée : Mesure de la Précision de l'Attribut (Attribute Disclosure Accuracy)**

L'objectif est de deviner une information sensible (ici `liste_diag`) avec certitude. Même si l'individualisation échoue (confusion entre deux personnes), l'inférence peut réussir si les deux personnes partagent la même maladie.

* **Méthodologie :**
    1.  **Lien par Quasi-Identifiants :** On lie les lignes de la base publiée à la base de référence via les variables connues.
    2.  **Comparaison de la valeur sensible :** On compare la valeur trouvée dans le fichier publié avec la vraie valeur dans le fichier de référence.
    3.  **Calcul de succès :** Une inférence est considérée comme réussie si la valeur est identique, *indépendamment de l'identifiant*.

### 3.3 Risque de Corrélation (Linkability / Structure)
**Technique utilisée : Comparaison de Matrices de Corrélation de Spearman**

Le mélange par colonne vise à casser la structure des données. Cet indicateur mesure à quel point cette structure a survécu.

* **Méthodologie :**
    1.  **Encodage :** Les variables catégorielles sont factorisées pour le calcul.
    2.  **Matrice de Spearman :** Nous calculons la corrélation de rang (Spearman) pour la base de référence ($M_{ref}$) et pour la base publiée ($M_{pub}$). *Spearman est préféré à Pearson pour les relations non linéaires et ordinales.*
    3.  **Différence Moyenne :** Nous calculons la différence absolue moyenne entre les deux matrices.
    4.  **Score :** $100 \times (1 - \text{Différence} \times 2)$. Si la différence est 0, le risque est 100% (structure intacte).

In [ ]:
def calculate_correlation_risk(df_ref, df_anon):
    """Calcule la conservation de la structure de corrélation (0-100%)."""
    common_cols = [c for c in df_ref.columns if c in df_anon.columns and c != 'id_sejour']
    if not common_cols: return 0.0
    
    # Encodage factoriel pour gérer les catégorielles et calcul Spearman
    def get_encoded_corr(df, cols):
        temp = pd.DataFrame()
        for c in cols:
            if df[c].dtype == 'object': temp[c] = pd.factorize(df[c])[0]
            else: temp[c] = df[c]
        return temp.corr(method='spearman')

    corr_ref = get_encoded_corr(df_ref, common_cols).fillna(0)
    corr_anon = get_encoded_corr(df_anon, common_cols).fillna(0)
    
    # Différence moyenne absolue
    mean_diff = (corr_ref - corr_anon).abs().mean().mean()
    
    # Score : 100% si identique, diminue si différence augmente
    return max(0, (1 - mean_diff * 2)) * 100

def calculate_risk_metrics(df_connaissance, df_publie, quasi_identifiers, sensitive_col='liste_diag'):
    """
    Calcule les 3 indicateurs : Individualisation, Inférence, Corrélation.
    """
    # 1. Corrélation (Globale)
    risk_corr = calculate_correlation_risk(df_connaissance, df_publie)

    # Vérification colonnes
    missing_cols = [c for c in quasi_identifiers if c not in df_publie.columns]
    if missing_cols:
        return 0.0, 0.0, risk_corr

    # 2. Individualisation
    # On groupe par QI pour trouver les uniques dans la base publiée
    counts = df_publie.groupby(quasi_identifiers).size()
    unique_combs = counts[counts == 1].index
    
    if len(unique_combs) == 0:
        return 0.0, 0.0, risk_corr
    
    df_pub_unique = df_publie.set_index(quasi_identifiers)
    df_pub_unique = df_pub_unique[df_pub_unique.index.isin(unique_combs)].reset_index()
    
    # Attaque par jointure avec la connaissance externe
    merged = pd.merge(
        df_pub_unique, 
        df_connaissance, 
        on=quasi_identifiers, 
        how='inner', 
        suffixes=('_pub', '_know')
    )
    
    if len(merged) == 0: return 0.0, 0.0, risk_corr
        
    # Succès si l'ID d'origine est retrouvé (Vérité terrain)
    reid_success = merged[merged['id_sejour_pub'] == merged['id_sejour_know']]
    risk_reid = len(reid_success) / len(df_publie) * 100
    
    # 3. Inférence
    # Si la variable sensible est présente dans la base, on compare sa valeur
    if sensitive_col in df_publie.columns:
        # Précision de la valeur sensible trouvée par l'attaquant
        inf_success = merged[merged[f'{sensitive_col}_pub'] == merged[f'{sensitive_col}_know']]
        risk_inference = len(inf_success) / len(merged) * 100
    else:
        risk_inference = 0.0
    
    return risk_reid, risk_inference, risk_corr

## 4. Exécution des Scénarios

Nous utilisons des scénarios de "connaissance croissante" incluant âges, dates, modes d'entrée/sortie et pathologies.

In [ ]:
scenarios = {
    '1_Faible': ["age10","sexe","entree_mode","sortie_mode","entree_date_y","sortie_date_y"],
    '2_Moyen': ["age5","sexe","entree_mode","sortie_mode","entree_date_ym","sortie_date_ym","specialite","chirurgie"],
    '3_Fort': ["age","sexe","entree_mode","sortie_mode","entree_date_ymd","sortie_date_ymd","specialite","chirurgie","diabete","insuffisance_renale","demence"]
}

results = []

# Cible pour l'inférence : 'liste_diag' (Diagnostics précis CIM10)
TARGET_INFERENCE = 'liste_diag' 

for name, df_var in datasets.items():
    if name == 'reference': continue
    
    parts = name.split('_')
    if len(parts) >= 3:
        file_type = 'Direct' if 'direct' in name else 'Sample'
        try: shuffle_level = int(parts[-1])
        except: shuffle_level = 0
    else: continue

    for scen_name, cols in scenarios.items():
        r_reid, r_inf, r_corr = calculate_risk_metrics(df_reference, df_var, cols, sensitive_col=TARGET_INFERENCE)
        
        results.append({
            'Type': file_type,
            'Mélange (%)': shuffle_level,
            'Scénario': scen_name,
            'Individualisation (%)': r_reid,
            'Inférence (%)': r_inf,
            'Corrélation (%)': r_corr
        })

df_res = pd.DataFrame(results)
print("Aperçu des résultats :")
display(df_res.head())

## 5. Visualisation et Analyse

Comparaison des risques (Individualisation, Inférence, Corrélation) selon le niveau de mélange et le scénario.

In [ ]:
def plot_all_metrics(df_res):
    if df_res.empty:
        print("Pas de données à afficher.")
        return

    fig, axes = plt.subplots(1, 3, figsize=(22, 6))
    
    # 1. Individualisation
    sns.lineplot(data=df_res, x='Mélange (%)', y='Individualisation (%)', 
                 hue='Scénario', style='Type', markers=True, ax=axes[0])
    axes[0].set_title("Risque d'Individualisation (Réidentification)")
    axes[0].set_ylim(-5, 105)

    # 2. Inférence
    sns.lineplot(data=df_res, x='Mélange (%)', y='Inférence (%)', 
                 hue='Scénario', style='Type', markers=True, ax=axes[1])
    axes[1].set_title(f"Risque d'Inférence (Cible: {TARGET_INFERENCE})")
    axes[1].set_ylim(-5, 105)

    # 3. Corrélation
    subset = df_res[df_res['Scénario'] == list(scenarios.keys())[0]]
    sns.lineplot(data=subset, x='Mélange (%)', y='Corrélation (%)', 
                 style='Type', markers=True, ax=axes[2], color='green')
    axes[2].set_title("Risque de Corrélation (Conservation Structure)")
    axes[2].set_ylim(-5, 105)

    plt.tight_layout()
    plt.show()

plot_all_metrics(df_res)

### Conclusion et Discussion
* **Individualisation** : Le scénario 3 (Fort), qui inclut des dates précises et des pathologies, montre un risque initial très élevé qui chute avec le mélange.
* **Inférence** : La capacité à prédire `liste_diag` dépend fortement de la qualité de la réidentification. Une fois l'individu perdu, l'inférence devient aléatoire.
* **Corrélation** : Cet indicateur montre à quel point la structure statistique de la base est détruite. Si le risque de corrélation est faible, la base perd aussi son utilité statistique pour les chercheurs.